In [5]:
def strcounter(s):
    slozhnost = 0
    for symbol in s:
        count = 0
        for sub_symbol in s:
            if symbol == sub_symbol:
                count += 1
            slozhnost += 1
        print(symbol, ":", count)
    return slozhnost
s = "aaabbbccddddddeeeeeee"
sl = strcounter(s)
n = len(s)
print(n, sl)

a : 3
a : 3
a : 3
b : 3
b : 3
b : 3
c : 2
c : 2
d : 6
d : 6
d : 6
d : 6
d : 6
d : 6
e : 7
e : 7
e : 7
e : 7
e : 7
e : 7
e : 7
21 441
